In [11]:
# !pip install --upgrade jax[cuda] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

/apps/pytorch/2.2.0/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached nvidia_cudnn_cu12-9.5.0.50-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.5.0.50-py3-none-manylinux2014_x86_64.whl (562.9 MB)
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.5.0.50 which is incompatible.


In [1]:
import os
import csv
import jax
import haiku as hk
import numpy as np
import pandas as pd

# import torch
# import pickle
# import swifter
# import seaborn as sns
# from tqdm import tqdm

import jax.numpy as jnp
import matplotlib.pyplot as plt

import multiprocessing as mp

from nucleotide_transformer.pretrained import get_pretrained_model

# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device

from datasets import Dataset, DatasetDict

%run preprocess_utility.py

print(jax.devices())


/home/sunhuaikuan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[CudaDevice(id=0)]


In [2]:
# !pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !pip install git+https://github.com/deepmind/dm-haiku


# import jax
# print(jax.devices())
# !pip install --upgrade pip

# !pip install opt_einsum
# !pip install swifter

In [2]:
try:
    import nucleotide_transformer
except:
    !pip install numpy==1.23.5
    !pip install git+https://github.com/instadeepai/nucleotide-transformer@main |tail -n 1
    import nucleotide_transformer

if "COLAB_TPU_ADDR" in os.environ:
    from jax.tools import colab_tpu

    colab_tpu.setup_tpu()

In [3]:
comp = {'A':1, 'C':2, 'G':3, 'T':4}

#@title Select a model
#@markdown ---
model_name = '50M_multi_species_v2'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2', '1B_agro_nt']
model_name = '500M_human_ref'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2', '1B_agro_nt']
#@markdown ---

# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    embeddings_layers_to_save=(20,),
    attention_maps_to_save=((1, 4), (7, 18)),
    max_positions=32,
    # If the progress bar gets stuck at the start of the model wieghts download,
    # you can set verbose=False to download without the progress bar.
    verbose=False
)
forward_fn = hk.transform(forward_fn)


In [5]:
# # Get data and tokenize it
# sequences = [
#     "ATTCCGAAATCGCTGACCGATCGTACGAAA",
#     "ATTTCTCTCTCTCTCTGAGATCGATCGATCGATATCTCTCGAGCTAGC",
# ]
# tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
# tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
# tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)

In [6]:
# !pip install jax jaxlib==0.1.87+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [4]:
fasta_file = "/blue/xiaofan/sunhuaikuan/gpn/examples/msa/genome.hg38rg.fa"
chrom_sequences = read_fasta(fasta_file)
# print(chrom_sequences.keys()) 

In [5]:
# datafile='clinvar_20240805.noncoding'
# datafile='clinvar_20240805.missense_matched'
datafile='methylation'

### input methylation datafile file, </br> output methylation_embedding.csv file

In [6]:
data_filename = '/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task05-methylation/GSM6637962_CpG_coverage20_GRCh38.bed.gz' 
df = preprocess_datafile(data_filename)
df

,CHROM,START,SIZE,y
0,1,788407,128,100
1,1,788472,128,80
2,1,788825,128,95
3,1,788844,128,90
4,1,788859,128,100
...,...,...,...,...
54789,Y,56870903,128,65
54790,Y,56870983,128,17
54791,Y,56871109,128,34
54792,Y,56871502,128,0


In [10]:
# %%time

# # sub_df = pd.DataFrame()    
# # final_df = pd.DataFrame()
# # segment=800


# # csv_Filename = './clinvar_20240805_noncoding_embeddings_nt.csv'
# # csv_Filename = './clinvar_20240805_missense_embeddings_nt.csv'
# # if os.path.exists(csv_Filename):
#     # os.remove(csv_Filename)

# max_length= 128 # 186



# df=pd.read_csv(datafile+'.txt', delimiter='\t')
# # df=pd.read_csv('clinvar_20240805.missense_matched.txt', delimiter='\t')

# columns_to_keep=['CHROM','POS','ID','REF','ALT','Pathogenicity']
# df = df[columns_to_keep]

# # Merge CHROM=9 and '9' etc

# for i in range(1,23):
#     df.loc[df['CHROM']==str(i),'CHROM']=i


# df['START']= df['POS']- max_length //2 -1
# df['END']  = df['START'] + max_length
# df['SIZE'] = max_length

# df=df[~df['CHROM'].isna()]

# Pathogenicity_dict={'B':0,'P':1}
# df['y'] = df['Pathogenicity'].map(Pathogenicity_dict)
# # df['CHROM'].value_counts()
# df.drop(['POS','ID','Pathogenicity','END'], axis=1, inplace=True)
# df

## Step 1: Obtain sequence section for all data and attach to dataframe df

In [10]:
%%time

import multiprocessing as mp


def get_sequencesection(row):

    chrom=row['CHROM']
    # pos_start=row['POS']-93
    pos_start=row['START']
    # ref=row['REF']
    # alt=row['ALT']        
    y=row['y']
    if row['SIZE'] % 6 == 0:
        length = row['SIZE']
    else:
        length = 6 * round(row['SIZE'] / 6)
    
    # chrom = 'chr' + chrom if chrom.isdigit() else chrom
    # subsequence = get_subsequence(str(chrom), pos_start, length)
    subsequence = chrom_sequences[str(chrom)][pos_start:pos_start + length]
    return subsequence


def parallelize_dataframe(df, func, num_partitions=100):
    # Split DataFrame into smaller chunks
    df_split = np.array_split(df, num_partitions)
    pool = mp.Pool(num_partitions)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Define a wrapper function to apply the get_sequencesection function
def apply_get_sequencesection(df):
    df['dna'] = df.apply(get_sequencesection, axis=1)
    return df


# df_top = df.head(1000)
df_with_dna = df
    
df_with_dna = parallelize_dataframe(df_with_dna, apply_get_sequencesection)

# Save the DataFrame 'df' to a CSV file
# df_with_dna.to_csv(datafile+'_with_dna.csv', index=False)
df_with_dna= df_with_dna.drop(columns=['CHROM','START','SIZE'])
df_with_dna

/home/sunhuaikuan/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/apps/pytorch/2.2.0/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/apps/pytorch/2.2.0/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


CPU times: user 123 ms, sys: 3.63 s, total: 3.75 s
Wall time: 8.88 s


,y,dna
0,100,CGATGAGAGACAGAGAGAAGGAGAGAGAAAGTACAAAAGAACGAAT...
1,80,CGAGCAGGAACCTTGGAGGACCTATTGCTTAAGGTGTGGGCCAAAG...
2,95,CGAATGGAATGGAACGGAACGGAATGGAATGGAACGCACTCGAATG...
3,90,CGGAATGGAATGGAACGCACTCGAATGGAATGGAACGGACATGAAT...
4,100,CGCACTCGAATGGAATGGAACGGACATGAATGGAATGGAATGGAAC...
...,...,...
54789,65,GGCCTCAAGGAATAGGAGTGCTGGGTCTATCAGTCTCTCCCAACCA...
54790,17,GAGCTCACTCTTGGATCAGATGGGGCCTCCCTGCCAGCAGTTCTAC...
54791,34,GGTGACTCAAACCTTCAGGCAGTGAAAATAAGGAGACTCACAAACT...
54792,0,GGCCCTGTGAACATGCTGGTGACAGTGTGAGCAGGCAGGGCTGGTT...


In [12]:
# df_top[df_top['START']==788407]['dna'].str.len()

In [13]:
num_parallel = 10
!export XLA_FLAGS=--xla_gpu_force_compilation_parallelism=num_parallel

/apps/pytorch/2.2.0/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


## THe below version complete noncoding 95760 in 1 min 29 second 

In [14]:
# %%time

# num_parallel = 20
# !export XLA_FLAGS=--xla_gpu_force_compilation_parallelism=num_parallel

# import dask.dataframe as dd
# import pandas as pd
# import jax.numpy as jnp
# from dask.diagnostics import ProgressBar

# # Vectorized tokenization function
# def vectorized_tokenizer(subsequences):
#     # Assuming tokenizer.batch_tokenize can handle a list of subsequences
#     tokens_ids = [b[1] for b in tokenizer.batch_tokenize(subsequences)]
#     return jnp.asarray(tokens_ids, dtype=jnp.int32)

# # Vectorized embedding function
# def vectorized_embedding(tokens):
#     random_key = jax.random.PRNGKey(0)
#     # Assuming forward_fn.apply() can handle batches of tokens
#     outs = forward_fn.apply(parameters, random_key, tokens)
#     return outs["embeddings_20"][:, 0, :]

# # Tokenization and embedding combined in a batch-wise function
# def process_batch(subsequences):
#     tokens = vectorized_tokenizer(subsequences)
#     embeddings = vectorized_embedding(tokens)
#     return embeddings

# # Function to apply on each Dask partition
# def apply_get_tokens_dask(df):
#     subsequences = df['dna'].values  # Get all subsequences in the batch
#     embeddings = process_batch(subsequences)  # Process in a vectorized manner
    
#     df['embedding'] = list(embeddings)  # Assign embeddings back to the DataFrame
#     # df['embedding'] =embeddings  # Assign embeddings back to the DataFrame
#     # embedding_df = pd.DataFrame(embeddings, columns=[str(i) for i in range(0, 1280)])
#     # df = pd.concat([df, embedding_df], axis=1)
    
#     return df

# # Load the CSV file into a Pandas DataFrame
# df = pd.read_csv(datafile + '_with_dna.csv')

# # For demonstration, process a small subset, but this will work for larger datasets
# # df =df.head(10000)

# # Convert the Pandas DataFrame to a Dask DataFrame
# ddf = dd.from_pandas(df, npartitions=num_parallel)  # Adjust 'npartitions' based on resources

# # Create metadata for Dask to understand the structure of the DataFrame
# meta = df.copy()
# meta['embedding'] = object  # Tokens will be of type object (adjust type accordingly)
# # meta['embedding'] = np.ndarray  # Tokens will be of type object (adjust type accordingly)

# # Apply the function in parallel using Dask
# ddf = ddf.map_partitions(apply_get_tokens_dask, meta=meta)

# # Compute the result with progress tracking
# with ProgressBar():
#     df_top = ddf.compute()

# df_top.to_csv(datafile+'_with_dna'+'_with_embedding.csv', index=False)
# df_top
# # print(df_top.head())


In [11]:
%%time

import dask.dataframe as dd
import pandas as pd
import jax.numpy as jnp
import numpy as np
from dask.diagnostics import ProgressBar

# Set number of parallel partitions
num_parallel = 10
batch_size = 10000  # Set batch size to 64

# Vectorized tokenization function
def vectorized_tokenizer(subsequences):
    tokens_ids = [b[1] for b in tokenizer.batch_tokenize(subsequences)]
    return jnp.asarray(tokens_ids, dtype=jnp.int32)

# Vectorized embedding function
def vectorized_embedding(tokens):
    random_key = jax.random.PRNGKey(0)
    outs = forward_fn.apply(parameters, random_key, tokens)
    return outs["embeddings_20"][:, 0, :]  # Return embeddings

# Tokenization and embedding combined in a batch-wise function
def truncate_sequences(sequences, max_length=32):
    """Truncate each sequence to the max_length."""
    return [seq[:max_length] for seq in sequences]

def process_batch(subsequences):
    truncated_sequences = truncate_sequences(subsequences, max_length=32)
    tokens = vectorized_tokenizer(truncated_sequences)  # Tokenize the subsequences
    embeddings = vectorized_embedding(tokens)  # Get embeddings
    return embeddings




# Function to apply on each Dask partition
def apply_get_tokens_dask(df):
    # Create an empty list to collect embeddings
    all_embeddings = []

    # Process subsequences in batches
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        subsequences = df['dna'].values[start:end]  # Get the current batch
        embeddings = process_batch(subsequences)  # Process the current batch
        all_embeddings.append(embeddings)

    # Concatenate all embeddings into a single array
    all_embeddings = jnp.concatenate(all_embeddings, axis=0)

    # Create a DataFrame from embeddings
    embedding_df = pd.DataFrame(all_embeddings, columns=[str(i) for i in range(1, 1281)])

    # Concatenate the original DataFrame with the embedding DataFrame
    df = pd.concat([df.reset_index(drop=True), embedding_df.reset_index(drop=True)], axis=1)

    return df

# Load the CSV file into a Pandas DataFrame
# df = pd.read_csv(datafile + '_with_dna.csv')
df = df_with_dna

# Convert the Pandas DataFrame to a Dask DataFrame
ddf = dd.from_pandas(df, npartitions=num_parallel)  # Adjust 'npartitions' based on resources

# Create metadata for Dask to understand the structure of the DataFrame
meta = df.copy()
# meta = meta.drop(columns=['dna'])  # Drop 'dna' if you don't need it in the final DataFrame
for i in range(1, 1281):
    meta[str(i)] = np.ndarray  # Prepare for 1280 embedding columns

# Apply the function in parallel using Dask
ddf = ddf.map_partitions(apply_get_tokens_dask, meta=meta)

# Compute the result with progress tracking
with ProgressBar():
    df_top = ddf.compute()

df_top= df_top.drop(columns=['dna'])
df_top= swapfirst2last(df_top)

# Save the results to CSV
df_top.to_csv(datafile + '_nt_embedding.csv', index=False)

# Display the DataFrame
print(df_top.head())

<timed exec>:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[                                        ] | 0% Completed | 776.54 ms

2024-10-31 16:33:20.194388: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.68. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


[                                        ] | 0% Completed | 2.22 s msBefore attention blocks: [[[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-1.2840297   0.4719833  -2.2279875  ... -0.1427693   0.6460447
   -2.1994483 ]
  [ 0.62872744  1.1597593   0.4775821  ...  1.3492723   0.6983742
   -1.6324239 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]]

 [[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-0.9324149   0.52861655 -1.6110799  ... -0.6671132   0.90874654
   -2.2312975 ]
  [-1.1377717   2.135611   -0.28993845 ... -0.16258729  0.6702845
   -0.18329185]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   

In [12]:
import pandas as pd

# datafile='clinvar_20240805.missense_matched'
df=pd.read_csv(datafile+'_nt_embedding.csv')
# df=df.head(100)
df

,1,2,3,4,5,6,7,8,9,10,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,y
0,0.014670,4.295811,-0.737905,6.260756,-10.438107,2.607501,-8.981725,4.586462,1.427386,31.805730,...,1.407268,-3.548049,-5.542783,6.938684,-6.547020,-16.476124,-17.860586,-8.432025,5.747534,100
1,-3.512577,9.073802,1.418918,3.949963,-5.045628,5.043728,-10.646418,10.077398,-0.885716,30.536760,...,-2.543087,-4.519937,-4.593312,3.605703,-4.628235,-18.188509,-17.656374,-4.776138,4.899363,80
2,-0.704749,5.617012,5.544855,7.722315,-7.870079,8.058717,-5.974363,6.684918,0.054571,34.394930,...,-0.771225,-6.872328,-4.710540,4.730622,-4.009584,-15.282115,-16.462130,-6.342956,7.509364,95
3,-0.338165,10.950527,3.851943,6.362251,-11.123697,4.273367,-7.971213,9.178355,0.551098,34.639774,...,0.414909,-7.359153,-2.953081,5.102181,-6.127724,-16.965920,-14.996564,-7.426395,4.208138,90
4,0.736067,8.545635,7.348443,7.398864,-11.428895,7.280689,-5.710442,8.959304,-1.290090,32.276160,...,0.988429,-8.099735,-2.944104,4.286998,-6.282523,-16.236626,-14.968699,-3.311508,9.740802,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47948,3.272444,12.186039,2.545142,5.892594,-7.051033,6.465103,-9.658198,6.110871,1.952545,33.166122,...,2.459701,-5.179332,-2.247317,4.099805,-2.065813,-22.586046,-15.919465,-3.994073,2.435006,65
47949,1.841231,8.751868,6.477094,6.309408,-7.666615,0.548814,-7.010179,6.689471,0.978388,25.027870,...,5.891490,-2.354367,-2.778426,9.645118,-5.097598,-19.130960,-12.292728,-4.416072,8.519820,17
47950,1.252075,9.786252,1.425042,7.753856,-6.902560,5.289457,-7.321176,9.746491,-0.111998,33.170880,...,2.000818,-5.716644,-2.034302,6.425899,-3.352080,-19.514114,-17.275158,-5.917955,5.764585,34
47951,-3.014114,12.801002,2.652979,13.168077,-9.895447,7.341707,-7.027656,11.894932,1.691979,34.484936,...,1.941798,-5.452161,1.353195,4.320939,-3.568731,-20.419058,-18.996656,-4.189190,0.456586,0


## Below codes are not used for now

### Do the work then save to hf_dataset

In [14]:
from datasets import Dataset, Features, Sequence, Value, load_from_disk ,concatenate_datasets # Add load_from_disk here
import tempfile

from datasets import Dataset, Features, Sequence, Value, load_from_disk, concatenate_datasets
import numpy as np
import jax.numpy as jnp

def FillHFset(rows, num_of_column, additional_columns, path_to_set):
    # Create column names (embedding columns) and extend with additional columns
    column_names = [str(i) for i in range(num_of_column)] + list(additional_columns.keys())
    
    # Create DataFrame with all rows and columns
    df = pd.DataFrame(rows, columns=column_names)
    
    # Create embedding columns (float sequences) only for numeric columns
    embedding_columns = {
        str(col): Sequence(Value("float32")) for col in range(num_of_column) if pd.api.types.is_numeric_dtype(df[str(col)])
    }
    
    # Handle non-numeric columns properly (e.g., cast strings as Value('string'))
    non_numeric_columns = {
        str(col): Value("string") for col in range(num_of_column) if pd.api.types.is_string_dtype(df[str(col)])
    }

    # Define all features (embedding + additional columns)
    features = Features({**embedding_columns, **non_numeric_columns, **additional_columns})

    # Create Hugging Face dataset and cast to features
    hf_dataset = Dataset.from_pandas(df).cast(features)
    
    # Check if dataset exists, load and concatenate if necessary, otherwise save new dataset
    if os.path.exists(path_to_set):
        existing_dataset = load_from_disk(path_to_set)
        combined_dataset = concatenate_datasets([existing_dataset, hf_dataset])

        # Use a temporary directory to save the combined dataset first
        with tempfile.TemporaryDirectory() as tmp_dir:
            combined_dataset.save_to_disk(tmp_dir)
            
            # Then replace the original dataset with the updated one
            shutil.rmtree(path_to_set)  # Remove the old dataset
            shutil.move(tmp_dir, path_to_set)  # Move new dataset to the original location

    else:
        hf_dataset.save_to_disk(path_to_set)
    return


def output2HFset(df, path_to_set):
    
    rows = []
    additional_columns = {
        'CHROM': Value("string"),                  # Chromosome names
        'REF': Value("string"),                    # Reference alleles
        'ALT': Value("string"),                    # Alternate alleles
        'SIZE': Value("int32"),                    # Region sizes
        'y': Value("int32")                        # Target labels
    }

    for index, row in df.iterrows():
        if ((index % 1000) == 0):
            print(f"index={index}")

        chrom = row['CHROM']
        ref = row['REF']
        alt = row['ALT']
        pos_start = row['START']
        pos_end = row['START'] + row['SIZE']
        y = row['y']

        try:
            tokens = row['tokens']


            # Step 1: Remove brackets and newline characters
            cleaned_str = tokens.replace('[', '').replace(']', '').replace('\n', '')
            
            # Step 2: Split the string by whitespace to get individual numbers as strings
            str_numbers = cleaned_str.split()
            
            # Step 3: Convert the list of string numbers into integers
            int_numbers = list(map(int, str_numbers))
            
            # Now you can convert this clean list to NumPy or JAX array
            import numpy as np
            tokens_np = np.array([int_numbers], dtype=np.int32)

            
            # Now it's safe to convert it to a JAX array
            import jax.numpy as jnp
            
            tokens_jax = jnp.asarray(tokens_np)

            
            embedding = get_embeddings(tokens_jax)  # Get embeddings

            # print("output2HFset : embedding.shape=:",embedding.shape)
            one_row = np.append(embedding, [chrom, ref, alt, pos_end - pos_start, y])
            # print("one_row=:",one_row)
            # print("one_row shape=:",one_row.shape)
            rows.append(one_row)


        except Exception as e:
            print(f"Exception caught: {e} for {row['CHROM']}-{row['START']}")
    
        # Every 5000 rows, save a batch to the dataset
        if ((index % 5000) == 0 and index > 0):
            num_of_column = embedding.shape[1]
            FillHFset(rows, num_of_column, additional_columns, path_to_set)
            rows = []
            print(f"Completed batch at index={index}")

    # If there are remaining rows after the loop, add them
    if rows:
        num_of_column = embedding.shape[1]
        FillHFset(rows, num_of_column, additional_columns, path_to_set)     


In [ ]:
%%time

import shutil

path_to_set="my_hf_datasets/20240805/noncoding_embeddings_nt"
# Remove the dataset directory and its contents
try:
    shutil.rmtree(path_to_set)
    print(f"Dataset at {path_to_set} has been removed.")
except Exception:
    # Silently catch the exception and do nothing
    pass
    
# df_top = df.head(35)
output2HFset(df, path_to_set)

Dataset at my_hf_datasets/20240805/noncoding_embeddings_nt has been removed.
index=0
Before attention blocks: [[[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [ 0.50158495  0.779868   -1.8287711  ... -1.2115262  -0.2792523
   -1.2028177 ]
  [ 0.2674231   0.92444116  1.2842956  ...  0.08013222 -0.18082626
   -1.1940216 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]]]


2024-09-25 17:28:21.862089: W external/xla/xla/service/gpu/gemm_fusion_autotuner.cc:836] Compiling 16 configs for gemm_fusion_dot.1 on a single thread.


Before attention blocks: [[[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-0.41093105  2.187459   -2.284892   ...  1.0488595   0.94676834
   -1.5457382 ]
  [ 1.6040431   2.4732795   1.469671   ...  0.9146555   0.5292439
   -1.1846138 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]]]
Before attention blocks: [[[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-1.6173527   1.8079523  -2.9425523  ... -0.67080766 -0.51572955
   -1.4554822 ]
  [-0.54203916  0.7917464   0.4183545  ...  0.20731087 -0.23180176
   -1.4138888 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1

In [26]:
from datasets import load_from_disk
import pandas as pd

# Load the dataset from disk
path_to_set="my_hf_datasets/20240805/noncoding_embeddings_nt"


hf_dataset = load_from_disk(path_to_set)

num_rows = len(hf_dataset)


print(f"Number of rows in the dataset: {num_rows}")

# Convert the first 10 rows to a pandas DataFrame
df_top_10 = pd.DataFrame(hf_dataset[:10])

# Display the DataFrame
print(df_top_10)

# Convert the Hugging Face dataset to a pandas DataFrame
# df_20 = hf_dataset.to_pandas()
# df_20
# df_top=df.head(2)
# df_top
# df_20.iloc[0]['embeddings']
# df_20['embeddings'][0]
# print(df_20.head())


Number of rows in the dataset: 100
             0           1             2          3           4           5  \
0    0.3425328  15.1762085      3.742885   10.84721  -13.183308   5.8762307   
1    2.2413845    16.28364     1.1982976   13.38946   -8.391577   3.2226942   
2  0.007799506   15.692189  -0.080317944  12.313036   -9.821446    4.898007   
3   -3.9048407   18.416681     2.7687669  12.427805  -7.1248636   3.6083035   
4   -3.9048407   18.416681     2.7687669  12.427805  -7.1248636   3.6083035   
5    -3.960939   15.503376     3.5796745  10.260619  -11.944419   4.7667713   
6   -1.0721788   14.264588     1.6912824   6.378506  -9.9909525   5.1121697   
7  -0.88614994   16.601736     3.6419735  8.6276045   -9.367528   5.9740396   
8   -0.8275161   15.347647  -0.032351688   9.915802  -10.042259  0.66772527   
9     4.567762   14.807265      2.861988   9.738922   -9.466186   5.9770746   

            6          7          8          9  ...        1275        1276  \
0  -7.8857293  1

In [ ]:
# print(tokenizer._tokens_to_ids.keys())
df[df['START']==93500052]

In [ ]:
def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    print(df.shape)
    # new_arr = data_array[:, :-4]
    # last_column = data_array[:, -1]
    # data_array = np.column_stack((new_arr, last_column))
    
    column_names = [f'{i}' for i in range(0, df.shape[1]-6)]
    column_names.extend(['CHROM', 'START', 'SIZE', 'REF', 'ALT',  'y'])
    
    df.columns = column_names
    return df

df = load_embedding_file(csv_Filename)

df

### Load CSV File

In [ ]:
def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    print(df.shape)
    # new_arr = data_array[:, :-4]
    # last_column = data_array[:, -1]
    # data_array = np.column_stack((new_arr, last_column))
    
    column_names = [f'{i}' for i in range(0, df.shape[1]-6)]
    column_names.extend(['CHROM', 'START', 'SIZE', 'REF', 'ALT',  'y'])
    
    df.columns = column_names
    return df

df = load_embedding_file('./Homo_sapiens_embeddings_nt.csv')

df

In [ ]:
# df1 = load_embedding_file('./gnomad.v4.1.exon.nt.csv')
# df1['y']=0

# df2 = load_embedding_file('./gnomad.v4.1.intergenic.nt.csv')
# df2['y']=1

# df3 = load_embedding_file('./gnomad.v4.1.proximity.nt.csv')
# df3['y']=2


# df = pd.concat([df1, df2,df3], ignore_index=True)
# df.to_csv('gnomad.v4.1.nt.csv', index=False)

In [ ]:
# df = load_embedding_file('./gnomad.v4.1.nt.csv')
# df

In [ ]:
# df[df['POS']==843234]

In [ ]:
df = load_embedding_file('./Homo_sapiens_embeddings.csv')
df

In [ ]:
import os

# Specify the path to your CSV file
file_path = 'clinvar_noncoding_embeddings_nt.csv'

# Get the file size in bytes
file_size = os.path.getsize(file_path)
file_size_kb = file_size / 1024  # size in KB
file_size_mb = file_size_kb / 1024  # size in MB

# Print the file size in a readable format (optional)
print(f"File size: {file_size} bytes")
print(f"File size: {file_size_kb:.2f} KB")
print(f"File size: {file_size_mb:.2f} MB")


In [ ]:

mport os

# Specify the path to your CSV file
file_path = 'clinvar_noncoding_embeddings_nt.csv'

# Get the file size in bytes
file_size = os.path.getsize(file_path)
file_size_kb = file_size / 1024  # size in KB
file_size_mb = file_size_kb / 1024  # size in MB

# Print the file size in a readable format (optional)
print(f"File size: {file_size} bytes")
print(f"File size: {file_size_kb:.2f} KB")
print(f"File size: {file_size_mb:.2f} MB")

In [ ]:
import csv

# Specify the path to your CSV file
file_path = 'clinvar_noncoding_embeddings_nt.csv'

# Open the file and count the rows
with open(file_path, 'r') as file:
    reader = csv.reader(file)
    row_count = sum(1 for row in reader)

# Print the number of rows
print(f"Number of rows: {row_count}")


In [ ]:
import haiku as hk
import jax
import jax.numpy as jnp
from nucleotide_transformer.pretrained import get_pretrained_model

# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name="500M_human_ref",
    embeddings_layers_to_save=(20,),
    max_positions=32,
)
forward_fn = hk.transform(forward_fn)

# Get data and tokenize it
sequences = ["ATTCCGATTCCGATTCCG", "ATTTCTCTCTCTCTCTGAGATCGATCGATCGAT"]
sequences = ["ATTTCTCTCTCTCTCTGAGATCGATCGATCGAT"]
tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)

# Initialize random key
random_key = jax.random.PRNGKey(0)

# Infer
outs = forward_fn.apply(parameters, random_key, tokens)

# Get embeddings at layer 20
# print(outs["embeddings_20"].shape)

my_embedding=outs["embeddings_20"][:,0,:]
    
print("my_embedding shape=",my_embedding.shape)
column_names = [f'{i}' for i in range(0, my_embedding.shape[1])]
embedding_df = pd.DataFrame(my_embedding, columns=column_names)
embedding_df

In [ ]:
random_key = jax.random.PRNGKey(0)
def get_embeddings(tokens):

    # Initialize random key
    
    # Infer
    outs = forward_fn.apply(parameters, random_key, tokens)    
    # print(outs.keys())
    # print(outs["embeddings_20"].shape)
    # outs["embeddings_20"]
    
    # my_embedding=outs["embeddings_20"][:,16,:]
    my_embedding=outs["embeddings_20"][:,0,:]
    
    my_embedding.shape
    column_names = [f'{i}' for i in range(0, my_embedding.shape[1])]
    embedding_df = pd.DataFrame(my_embedding, columns=column_names)
    return embedding_df

In [22]:
import numpy as np
import pandas as pd

# Example list of embeddings (ensure these are numpy arrays)
embeddings = [np.random.rand(10) for _ in range(5)]  # List of 5 numpy arrays of size 10
embeddings
# len(embeddings)


# # Create a DataFrame
# df = pd.DataFrame()

# # Assign the embeddings directly to the DataFrame
# df['embedding'] = embeddings  # This will keep them as numpy arrays

# df.to_csv('demo.csv')


[array([0.82661685, 0.013857  , 0.05041127, 0.16914426, 0.27685434,
        0.48058288, 0.76627826, 0.43385749, 0.41895577, 0.97018492]),
 array([0.68989487, 0.99791568, 0.68382645, 0.59061445, 0.97438343,
        0.93641663, 0.36037968, 0.48704185, 0.35121413, 0.48548722]),
 array([0.13053355, 0.74875501, 0.77265241, 0.93544951, 0.48985972,
        0.43907184, 0.02972745, 0.3836498 , 0.53054333, 0.88076955]),
 array([0.75930673, 0.11030291, 0.627487  , 0.3309788 , 0.32840613,
        0.75120948, 0.59324971, 0.52827394, 0.66185284, 0.71089183]),
 array([0.73636719, 0.28920172, 0.35876011, 0.6249102 , 0.93867097,
        0.80249523, 0.98652413, 0.63058877, 0.72700444, 0.3312359 ])]